Technical Neural Networks

WS 2024/25

Assignment Group A

Can Tosun (2792747), Christopher Overdiek (3122952),
Zyad Altahan (50238152), Isra Zaman (50203794)

Programming Assignment E

In [62]:
import numpy as np
import matplotlib.pyplot as plt
import fileinput

rng = np.random.default_rng()

In [82]:
#Load training data and recall data from file and set size K
p = []
with fileinput.input(files="PA-E_train_1.txt", encoding="utf-8") as f:
    for line in f:
        if f.filelineno() > 1:
            p.append(list(map(int, line.split())))

K = len(p[0])
p = np.array(p)

with fileinput.input(files="PA-E_recall_1.txt", encoding="utf-8") as f:
  it = iter(f)
  next(it)
  r = list(map(int, next(it).split()))

r = np.array(r)

In [83]:
theta = np.zeros((K))

def learn(p):
  w = np.zeros((K, K))

  for q in p:
    v = np.outer(q, q)
    np.fill_diagonal(v, 0)
    assert np.array_equal(v.T, v) #v is symmetric
    w += v

  return w

def recall(w, input):
  X = input
  t = 0
  for _ in range(1000): #to prevent infinite loop with while(True)
    print(f"X(t = {t}): ", end='')
    printOutput(X)
    t += 1

    Xt_plus1 = X.copy()
    for k in np.random.permutation(K): #random order of asynchronous updates
      if np.dot(w[k], X) > theta[k]:
        Xt_plus1[k] = 1
      else:
        Xt_plus1[k] = -1
    if np.array_equal(Xt_plus1, X):
      break
    X = Xt_plus1

  return X

def printOutput(a):
  if K < 101:
    for i in a:
      if i == 1:
        print('+', end='') #end='' prevents newline
      else:
        print('-', end='')
    print('')
  else:
    #print with pyplot
    return

w = learn(p)
result = recall(w, r)
print("Result: ", end='')
printOutput(result)

X(t = 0): ++-++++---
X(t = 1): ++++++++++
Result: ++++++++++
